In [1]:
# import pygame
import random
from enum import Enum
from collections import namedtuple
import numpy as np

# pygame.init()
# font = pygame.font.Font('arial.ttf', 25)

class Direction(Enum):
    RIGHT = 1
    LEFT = 2
    UP = 3
    DOWN = 4

Point = namedtuple('Point', 'x, y')

# rgb colors
WHITE = (255, 255, 255)
RED = (200,0,0)
BLUE1 = (0, 0, 255)
BLUE2 = (0, 100, 255)
BLACK = (0,0,0)

BLOCK_SIZE = 20
SPEED = 400

class SnakeGame:
    def __init__(self, width, height):
        self.map = np.zeros((width, height))
        self.width = width
        self.height = height
        # init display
        # self.display = pygame.display.set_mode((self.w, self.h))
        # pygame.display.set_caption('Snake')
        # self.clock = pygame.time.Clock()
        self.reset()

    def reset(self):
        self.direction = Direction.RIGHT
        self.head = Point(self.width // 2, self.height // 2)
        
        self.snake = [
            self.head,
            Point(self.head.x - 1, self.head.y),
            Point(self.head.x - 2, self.head.y)
        ]
        self.map[self.head.x, self.head.y] = 1
        for body in self.snake[1:]:
            self.map[body.x, body.y] = 2
        self.score = 0
        self.food = None
        self._place_food()
        self.frame_iteration = 0

    def _place_food(self):
        x = random.randint(0, self.width - 1) 
        y = random.randint(0, self.height - 1)
        self.food = Point(x, y)
        if self.food in self.snake:
            self._place_food()
        else:
            self.map[self.food.x, self.food.y] = 3

    def play_step(self, action):
        self.frame_iteration += 1

        # 2. move
        self._move(action) # update the head
        self.snake.insert(0, self.head)

        # 3. check if game over
        reward = 0
        game_over = False
        if self.is_collision() or self.frame_iteration > 100*len(self.snake):
            game_over = True
            reward = -10
            return reward, game_over, self.score
        
        self.map[self.head.x, self.head.y] = 1
        
        # 4. place new food or just move
        if self.head == self.food:
            self.score += 1
            reward = 10
            self._place_food()
        else:
            tail = self.snake.pop()
            self.map[tail.x, tail.y] = 0

        # 6. return game over and score
        return reward, game_over, self.score

    def is_collision(self, pt=None):
        if pt is None:
            pt = self.head
        # hits boundary
        if pt.x >= self.width or pt.x < 0 or pt.y >= self.height or pt.y < 0:
            return True
        # hits itself
        if pt in self.snake[1:]:
            return True

        return False
    
    def render(self):
        for h in range(self.height + 2):
            for w in range(self.width + 2):
                p = Point(w - 1, h - 1)
                if w == 0 or w == self.width + 1 or h == 0 or h == self.height + 1:
                    print("#", end = "\n" if w == self.width + 1 else "")
                elif self.map[p.x, p.y] == 1:
                    print("@", end="")
                elif self.map[p.x, p.y] == 2:
                    print("O", end="")
                elif self.map[p.x, p.y] == 3:
                    print("*", end="")
                else:
                    print(" ", end="")

    def _move(self, action):
        # [straight, right, left]

        clock_wise = [Direction.RIGHT, Direction.DOWN, Direction.LEFT, Direction.UP]
        idx = clock_wise.index(self.direction)

        if np.array_equal(action, [1, 0, 0]):
            new_dir = clock_wise[idx] # no change
        elif np.array_equal(action, [0, 1, 0]):
            next_idx = (idx + 1) % 4
            new_dir = clock_wise[next_idx] # right turn r -> d -> l -> u
        else: # [0, 0, 1]
            next_idx = (idx - 1) % 4
            new_dir = clock_wise[next_idx] # left turn r -> u -> l -> d

        self.direction = new_dir

        x = self.head.x
        y = self.head.y
        self.map[x, y] = 2
        if self.direction == Direction.RIGHT:
            x += 1
        elif self.direction == Direction.LEFT:
            x -= 1
        elif self.direction == Direction.DOWN:
            y += 1
        elif self.direction == Direction.UP:
            y -= 1

        self.head = Point(x, y)


In [2]:
game = SnakeGame(16, 9)
game.render()

##################
#*               #
#                #
#                #
#                #
#      OO@       #
#                #
#                #
#                #
#                #
##################


In [21]:
action = np.array([1, 0, 0])
# action = np.array([0, 1, 0])
# action = np.array([0, 0, 1])

result = game.play_step(action)
print(result)
game.render()

(0, False, 0)
##################
#*               #
#@OO             #
#                #
#                #
#                #
#                #
#                #
#                #
#                #
##################
